In [ ]:
# импортируем необходимые модули
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy import integrate, optimize
from sklearn import preprocessing, model_selection, metrics
from sklearn.cluster import KMeans
# from kneed import KneeLocator не робатает пока у меня почему-то
from IPython.display import display

In [ ]:
%matplotlib inline

In [ ]:
# загружаем данные из файла в формате CSV, указываем символ-разделитель и сохраняем данные в дата-фрейм (переменную df)
df = pd.read_csv('../data/marketing_campaign.csv',sep='\t')
# проверяем, что данные загрузились - выводим первые несколько строк таблицы
df.head()

## Признаки:
- ID - уникальный идентификатор клиента 
- Year_Birth - год рождения клиента 
- Education - уровень образования  
- Marital_Status - семейное положение  
- Income - годовой доход семьи клиента 
- Kidhome - количество детей до 12 лет в семье
- Teenhome - количество подростков 13-18 лет в семье
- Dt_Customer - дата регистрации клиента
- Recency - количество дней с момента последней покупки
- MntWines - траты на вино
- MntFruits - траты на фрукты
- MntMeatProducts - траты на мясо
- MntFishProducts - траты на рыбу
- MntSweetProducts - траты на сладости
- MntGoldProds - траты на золото (ювелирные изделия)
- NumDealsPurchases - покупки со скидкой
- NumWebPurchases - покупки через сайт
- NumCatalogPurchases - покупки по каталогу
- NumStorePurchases - покупки в магазине
- NumWebVisitsMonth - количество посещений сайта в месяц
- AcceptedCmp1 - участие в 1-й кампании 
- AcceptedCmp2 - участие во 2-й кампании 
- AcceptedCmp3 - участие в 3-й кампании 
- AcceptedCmp4 - участие в 4-й кампании 
- AcceptedCmp5 - участие в 5-й кампании 
- Complain - подавал ли жалобу за последние 2 года 
- Z_CostContact - техническая колонка 
- Z_Revenue - техническая колонка 
- Response - отклик на предложение в последней кампании (1 - согласился, 0 - отказался)

In [ ]:
# объём данных (строк и столбцов - объектов и признаков)
df.shape

In [ ]:
# выводим типы данных
df.dtypes

In [ ]:
# целевой признак Response

In [ ]:
print(df.describe())

In [ ]:
df.info()

In [ ]:
# имена колонок в виде списка
df.columns

In [ ]:
# проверка на отсутствующие значения для всех колонок датафрейма
df.isnull().sum()


In [ ]:
# в ходе проверки данных обнаружилось, что в колонке Income есть пропуски


In [ ]:
# посмотрел колонки на дубликаты
df.duplicated().sum()

In [ ]:
df.nunique()

In [ ]:
# создаём копию датафрейма для дальнейшего анализа и преобразований, чтобы не изменять исходные данные
df_eda = df.copy()

In [ ]:
# удаляем колонки Z_CostContact, Z_Revenue так как они используются для расчета затрат и  не несут полезной информации для анализа
drop_cols = ['Z_CostContact', 'Z_Revenue']
df_eda = df_eda.drop(columns=[col for col in drop_cols if col in df_eda.columns])

In [ ]:
# Общие траты
spending_cols = ['MntWines', 'MntFruits', 'MntMeatProducts', 
                 'MntFishProducts', 'MntSweetProducts', 'MntGoldProds']
df_eda['Total_Spending'] = df_eda[spending_cols].sum(axis=1)

In [ ]:
# Общие покупки
purchase_cols = ['NumDealsPurchases', 'NumWebPurchases', 
                 'NumCatalogPurchases', 'NumStorePurchases']
df_eda['Total_Purchases'] = df_eda[purchase_cols].sum(axis=1)

In [ ]:
# Средний чек
df_eda['Avg_Check'] = df_eda['Total_Spending'] / df_eda['Total_Purchases'].replace(0, np.nan)


In [ ]:
# Участие в кампаниях
campaign_cols = ['AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 
                 'AcceptedCmp4', 'AcceptedCmp5']
df_eda['Total_Accepted_Cmp'] = df_eda[campaign_cols].sum(axis=1)
df_eda['Accepted_Any'] = (df_eda['Total_Accepted_Cmp'] > 0).astype(int)

In [ ]:
plt.figure(figsize=(10, 6))
plt.hist(df_eda['Income'].dropna(), bins=30, edgecolor='black', alpha=0.7, color='green')
plt.title('Распределение дохода', fontsize=14)
plt.xlabel('Доход')
plt.ylabel('Частота')
plt.show()

In [ ]:
# Средние траты по категориям
avg_spending = df_eda[spending_cols].mean().sort_values(ascending=False)

plt.figure(figsize=(12, 6))
bars = plt.bar(avg_spending.index, avg_spending.values, color='skyblue', edgecolor='black')
plt.title('Средние траты по категориям', fontsize=14)
plt.xlabel('Категория')
plt.ylabel('Средние траты')
plt.xticks(rotation=45)

# Добавляем значения на столбцы
for bar in bars:
    height = bar.get_height()
    plt.text(bar.get_x() + bar.get_width()/2., height,
             f'{height:.0f}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

print("\nСредние траты по категориям:")
print(avg_spending)